# Create user oriented table

Change rating data format

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

## Preprocessing data

In [25]:
ml_rating_filename = "ratings_re2.csv"
ml_ratings = pd.read_csv(ml_rating_filename, names=["userId", "movieId", "rating", "timestamp"])
ml_ratings = ml_ratings[["userId", "movieId", "rating"]]

# Groupby -> mean, 
ml_ratings['rating'] = ml_ratings.groupby(['userId', 'movieId'])['rating'].transform('mean') 
ml_ratings = ml_ratings.drop_duplicates()

ml_ratings.to_csv("ratings_re.csv", index=False, header=False)

some duplicate occur !!!! :(

### Change pivot to user centric

In [26]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

In [27]:
movie_list = open("moviesIdx2.txt").readlines()

In [28]:
user_c = CategoricalDtype(sorted(ml_ratings.userId.unique()), ordered=True)
# movie_c = CategoricalDtype(sorted(ml_ratings.movieId.unique()), ordered=True)
movie_c = CategoricalDtype([i for i in range(0, len(movie_list))]) #some movie has no rating at all

In [29]:
# Preparation to make pivot table

row = ml_ratings.userId.astype(user_c).cat.codes
col = ml_ratings.movieId.astype(movie_c).cat.codes
sparse_matrix = csr_matrix((ml_ratings["rating"], (row, col)),
                           shape=(user_c.categories.size, movie_c.categories.size))

In [30]:
sparse_matrix

<138493x15440 sparse matrix of type '<class 'numpy.float64'>'
	with 14094614 stored elements in Compressed Sparse Row format>

### find list of movie without rating

In [31]:
no_ratings = []

for i in range(0, 15440):
    if i not in movie_c.categories:
        no_ratings.append(i)
        
no_ratings

[]

### Remove rating > 5

In [32]:
sparse_matrix.max()

5.0

In [33]:
sparse_matrix[sparse_matrix > 5.0] = 0

In [34]:
sparse_matrix.max()

5.0

### Normalize

sparse_matrix = sparse_matrix.multiply(0.2)

## Save sparse CSR

In [35]:
# Save it
import pickle

filename = "ratings.csr"
pickle.dump(sparse_matrix, open(filename, 'wb'))

---

## Make Dataframe version

In [ ]:
ratings_pivot = pd.SparseDataFrame(sparse_matrix,
                         index=user_c.categories,
                         columns=movie_c.categories,
                         default_fill_value=0)

In [ ]:
ratings_pivot.head()

## Save dataframe

In [ ]:
# Save it
import pickle

filename = MOVIELENS_DATASET + "ratings.df"
pickle.dump(ratings_pivot, open(filename, 'wb'))